<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum Computer Vision</h1>

<h2>Imports<h2>

In [71]:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import requests
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import zipfile

**Gebruik een <i>environment variable</i> voor het instellen van het logleven van TensorFlow**

In [72]:
os.environ['TF_CPP_MIN_LEVEL'] = '2'

<h2>Data Collection</h2>

<h3>Data downloaden</h3>

In [73]:
dataUrl = requests.get("http://madm.dfki.de/files/sentinel/EuroSAT.zip")
dataFile = zipfile.ZipFile(io.BytesIO(dataUrl.content))

In [74]:
# get file names
txtFiles = list()

for txtFile in dataFile.namelist():
    txtFiles.append(txtFile)
    # print(txtFile)

<h3>Data filteren</h3>

In [75]:
txtFiles = [x for x in txtFiles if ".jpg" in x]

<h3>Data inlezen</h3>

In [76]:
imageObjects = np.zeros([len(txtFiles), 64, 64, 3])

i = 0

for pic in txtFiles:
    imageObjects[i] = np.asarray(Image.open(dataFile.open(pic))).astype('uint8')/255
    i += 1
    
print(imageObjects[0])    

[[[0.21176471 0.29803922 0.35294118]
  [0.21176471 0.29803922 0.35294118]
  [0.18823529 0.2627451  0.32941176]
  ...
  [0.32941176 0.36470588 0.39215686]
  [0.21960784 0.30980392 0.33333333]
  [0.18823529 0.30196078 0.3254902 ]]

 [[0.21960784 0.29019608 0.34509804]
  [0.22352941 0.29411765 0.34901961]
  [0.18823529 0.2627451  0.32156863]
  ...
  [0.31764706 0.36862745 0.4       ]
  [0.21568627 0.31372549 0.33333333]
  [0.18823529 0.30196078 0.3254902 ]]

 [[0.30196078 0.35294118 0.38823529]
  [0.29803922 0.35294118 0.38823529]
  [0.25490196 0.31764706 0.36862745]
  ...
  [0.23137255 0.32156863 0.34509804]
  [0.2        0.30588235 0.33333333]
  [0.19215686 0.30980392 0.34117647]]

 ...

 [[0.21568627 0.30588235 0.32941176]
  [0.21568627 0.30588235 0.32941176]
  [0.22745098 0.31764706 0.34117647]
  ...
  [0.28235294 0.36078431 0.36470588]
  [0.34509804 0.4        0.40392157]
  [0.32156863 0.36862745 0.36862745]]

 [[0.23529412 0.33333333 0.34901961]
  [0.23529412 0.33333333 0.34901961]


**Gebruik de bestandsnamen voor het verkrijgen van de labels**

In [77]:
imageLabels = np.empty(len(txtFiles), dtype = 'S20')

i = 0

for label in txtFiles:
    txtFiles[i] = label.split('/')[1]
    i += 1
    
print(txtFiles[1])    
print(len(imageLabels))

River
27000


In [81]:
# change them to integers in alphabetical order
labelNames, labelNumbers = np.unique(txtFiles, return_inverse=True)
labelDict = dict(zip(np.unique(labelNumbers), labelNames))
# print(labelNames)
print("labelNumbers", labelNumbers)
print("LabelNumbers Length", len(labelNumbers))


print("imageObjects", len(imageObjects))

np.array(np.unique(labelNumbers, return_counts=True)).T

labelNumbers [8 8 8 ... 6 6 6]
LabelNumbers Length 27000
imageObjects 27000


array([[   0, 3000],
       [   1, 3000],
       [   2, 3000],
       [   3, 2500],
       [   4, 2500],
       [   5, 2000],
       [   6, 2500],
       [   7, 3000],
       [   8, 2500],
       [   9, 3000]], dtype=int64)

<h3>Data splitten</h3>

In [82]:
trainSet, testSet, trainLabels, testLabels = train_test_split(imageObjects, labelNumbers, stratify = labelNumbers, train_size = 0.75, random_state=42)
print(trainSet.shape)
nrOfImages = len(trainSet)

print (len(trainSet), len(testSet),len(trainLabels), len(testLabels))

(20250, 64, 64, 3)
20250 6750 20250 6750


<h3>Data inspecteren</h3>

In [ ]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        number = i
        plt.imshow(trainSet[number])
        plt.xlabel(labelNames[trainLabels[number]]) 
    
    plt.show()

In [ ]:
inspectData()    

<h2>Maak een model</h2>

<h3>Definieer de netwerktopologie</h3>

<h3>Exercise 1</h3>
<p>
    Vul de netwerktopologie tot een bruikbaar Neural Netwerk onstaat.
</p>

In [ ]:
model =  keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))







<h3>Definieer de trainparameters</h3>

In [ ]:
lossFunction = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gdAlgorithm = keras.optimizers.Adam(learning_rate=0.001)
nrOfEpochs = 5

<h3>Train het model</h3>

In [ ]:
print(len(imageObjects))
print(len(trainLabels))
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")
history = model.fit(trainSet, trainLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)


<h3>Test het model</h3>

<h3>Exercise 2</h3>
<p>
    Test het Neural Netwerk met de testset en laat zien of het in staat is de juiste class te vinden.
</p>

In [ ]:
history_test = model.fit(testSet, testLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)


In [ ]:

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history_test.history['accuracy'], label = 'accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(testSet,  testLabels, verbose=2)

example_batch = trainSet[:10]
example_result = model.predict(example_batch)
example_result


